### ספריות


In [1]:
import pandas as pd
import numpy as np
import os
import sys

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
cwd = os.getcwd()

education_folder_path = os.path.dirname(cwd)

sys.path.append(education_folder_path)

### פונקציות גלובליות


In [3]:
from global_functions import remove_spaces_in_columns, up_load_df, first_15_division, equal_distribution_students, split_to_layers

### העלאת טבלאות


In [ ]:
# בתי ספר וגנים מעיריית ירושלים
JLM=up_load_df(r'{}\background_files\jerusalem_muni'.format(education_folder_path),'מוסדות בירושלים 2020')
JLM=remove_spaces_in_columns(JLM)

In [ ]:
# קאורדינטות של מוסדות חינוך במרחב ירושלים
JTMT_setls_schools_coordinates_with_src=up_load_df(r'{}\background_files'.format(education_folder_path),'JTMT_setls_schools_coordinates_with_src')
JTMT_setls_schools_coordinates_with_src=remove_spaces_in_columns(JTMT_setls_schools_coordinates_with_src)

### עיבוד


In [6]:
JLM.rename(columns={'סמל_חינוך': 'סמל_מוסד'}, inplace=True)

JLM = JLM.drop(columns=['פיקוח', 'מעמד_משפטי', "מס'_תלמידים_ז-ט", "מס'_תלמידים_י-יד", 'קוד_אזור_סטיסטי', 'תאור_אזור_סטטיסטי', 'Unnamed:_18', 'Unnamed:_19'])

# מחיקת שורות של גני ילדים
JLM=JLM[JLM['שלב_חינוך'] != 'גני ילדים']

In [7]:
# מחיקת שורות שבהן העמודה "סהכ_תלמידים" שווה לאפס
JLM=JLM[JLM['סה"כ_תלמידים'] != 0]

In [8]:
# בדיקה אם יש מוסדות עם אותו סמל מוסד
is_unique = JTMT_setls_schools_coordinates_with_src['סמל_מוסד'].is_unique

In [9]:
# יצירת טבלה של כל המוסדות בלי קוארדינטות
rows_with_zero = JLM[JLM['coordinate_x'] == 0]

# יצירת טבלה של כל המוסדות עם קוארדינטות
JLM = JLM[JLM['coordinate_x'] != 0]

In [10]:
# הגדרת מקור קוארדינטות ל- jerusalem_muni
JLM['SRC'] = 'jerusalem_muni'

In [11]:
# הוספת עמודת SRC לטבלה של כל המוסדות בלי קוארדינטות
rows_with_zero['SRC'] = ''

In [12]:
# הוספת קוארדינטות לטבלה של כל המוסדות בלי קוארדינטות
rows_with_zero = pd.merge(rows_with_zero, JTMT_setls_schools_coordinates_with_src[['סמל_מוסד', 'coordinate_x', 'coordinate_y', 'SRC']],
                     on='סמל_מוסד', how='left')

# עדכון העמודות coordinate_x, coordinate_y, SRC
rows_with_zero['coordinate_x'] = rows_with_zero['coordinate_x_y']
rows_with_zero['coordinate_y'] = rows_with_zero['coordinate_y_y']
rows_with_zero['SRC'] = rows_with_zero['SRC_y']

rows_with_zero = rows_with_zero.drop(['coordinate_x_x', 'coordinate_y_x', 'coordinate_x_y', 'coordinate_y_y', 'SRC_x', 'SRC_y'], axis=1)

In [13]:
# מיזוג הטבלאות של כל המוסדות עם קוארדינטות אחרי עדכון הטבלה של כל המוסדות בלי קוארדינטות
JLM=pd.concat([JLM, rows_with_zero], ignore_index=True)

In [14]:
# שינוי סמל_עירייה ל-סמל_מוסד
JLM['סמל_מוסד'] = JLM['סמל_עירייה']
JLM.drop(columns='סמל_עירייה', inplace=True)

In [15]:
# בדיקה אם יש מוסדות עם אותו סמל מוסד
is_unique = JLM['סמל_מוסד'].is_unique

In [16]:
# הגדרת רשימת הכיתות לטווח א עד יד'
class_map = ['א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'יא', 'יב', 'יג', 'יד']

# הגדרת עמודות לכיתות עם NaN כערך ברירת מחדל
for single_class in class_map:
    JLM[f'כיתה_{single_class}'] = np.nan

# פונקציה לחלוקה ראשונית של 15 תלמידים לכל כיתה בטווח
JLM = JLM.apply(lambda row: first_15_division(class_map, row), axis=1)
# JLM = JLM.apply(first_15_division(class_map), axis=1)

In [17]:
# פונקציה לחלוקת התלמידים הנותרים שווה בשווה
JLM = JLM.apply(lambda row: equal_distribution_students(class_map, row), axis=1)
# JLM = JLM.apply(equal_distribution_students, axis=1)

In [18]:
# הגדרת רשימת הכיתות לטווח א עד יד'
class_map = ['כיתה_א', 'כיתה_ב', 'כיתה_ג', 'כיתה_ד', 'כיתה_ה', 'כיתה_ו', 'כיתה_ז', 'כיתה_ח', 'כיתה_ט', 'כיתה_י', 'כיתה_יא', 'כיתה_יב', 'כיתה_יג', 'כיתה_יד']

# בדיקה בכל כיתה אם מספר התלמידים קטן מ-15 ועדכון ל-0
for single_class in class_map:
    JLM[single_class] = JLM[single_class].apply(lambda x: x if pd.isna(x) or x >= 15 else 0)

In [19]:
# חלוקה לשכבות גיל
ages = {
    'ele': ['כיתה_א', 'כיתה_ב', 'כיתה_ג', 'כיתה_ד', 'כיתה_ה', 'כיתה_ו'],
    'mid': ['כיתה_ז', 'כיתה_ח', 'כיתה_ט'],
    'high': ['כיתה_י', 'כיתה_יא', 'כיתה_יב', 'כיתה_יג', 'כיתה_יד']
}

# חישוב מספר התלמידים לכל שכבת גיל
JLM = split_to_layers(JLM, ages)

# חלוקה למגזרים
JLM.loc[JLM['אגף']=='ערבי','sector'] = 1
JLM.loc[JLM['אגף']=='חרדי','sector'] = 2
JLM.loc[JLM['אגף']=='כללי','sector'] = 3

# שינוי שם של העמודות
JLM.rename(columns={'סמל_מוסד': 'ID'}, inplace=True)
JLM.rename(columns={'coordinate_x': 'x'}, inplace=True)
JLM.rename(columns={'coordinate_y': 'y'}, inplace=True)

JLM = JLM.drop(['אגף', 'שלב_חינוך', 'כיתה_א', 'כיתה_ב', 'כיתה_ג', 'כיתה_ד', 'כיתה_ה', 'כיתה_ו', 'כיתה_ז', 'כיתה_ח', 'כיתה_ט', 'כיתה_י', 'כיתה_יא', 'כיתה_יב', 'כיתה_יג', 'כיתה_יד', 'מספר_כיתות', 'סה"כ_תלמידים', 'מכיתה', 'עד_כיתה', 'שם_מוסד', 'סוג_חינוך'], axis=1)

### ייצוא


In [20]:
JLM.to_excel(r'{}\Intermediates\JLM_final.xlsx'.format(cwd), index=False)